In [5]:
import redis

r = redis.Redis(host="localhost", port=6379, decode_responses=True)

In [6]:
ts = r.ts()

In [2]:
import os

DATABASE = os.getenv("DATABASE", "mtg")
DATABASE_HOST = "localhost"
DATABASE_USER = os.getenv("DATABASE_USER", "root")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD", "root")
DATABASE_PORT = os.getenv("DATABASE_PORT", "27017")

from pymongo import MongoClient

client = MongoClient(
    f"mongodb://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}"
)
db = client[DATABASE]
edhrec_daily = db["edhrec_daily"]
card_stocks_daily = db["card_stocks_daily"]

In [3]:
pricing = card_stocks_daily.find_one()
print(pricing)

{'date': datetime.datetime(2025, 3, 4, 10, 21, 46, 573000), 'card_id': '0000419b-0bba-4488-8f7a-6194544ce91e', 'prices': {'usd': '0.25', 'usd_foil': '0.42', 'usd_etched': None, 'eur': None, 'eur_foil': None, 'tix': '0.03'}, '_id': ObjectId('67c76bd1a8094fa6547cb654')}


In [29]:
ranking = edhrec_daily.find_one()
print(ranking)

{'date': datetime.datetime(2025, 2, 26, 10, 24, 17, 718000), 'card_id': '0000419b-0bba-4488-8f7a-6194544ce91e', '_id': ObjectId('67bf8453c71f970d104c125d'), 'edhrec_rank': None}


In [ ]:
from tqdm import tqdm

ts_key = "{card_id}:ts:edhrec"
rankings = edhrec_daily.find(sort=[("date", 1)])
for ranking in tqdm(rankings):
    if ranking["edhrec_rank"]:
        ts.add(
            ts_key.format(card_id=ranking["card_id"]),
            int(ranking["date"].timestamp() * 1000),
            ranking["edhrec_rank"],
            on_duplicate="LAST",
        )

1014603it [01:49, 9271.23it/s] 


ResponseError: TSDB: Error at upsert, update is not supported when DUPLICATE_POLICY is set to BLOCK mode

In [ ]:
from tqdm import tqdm

# 'prices': {'usd': '0.25', 'usd_foil': '0.42', 'usd_etched': None, 'eur': None, 'eur_foil': None, 'tix': '0.03'}

ts_key = "{card_id}:ts:pricing:{currency}"
pricing = card_stocks_daily.find(sort=[("date", 1)])
for pricing in tqdm(pricing):
    if pricing["prices"]["usd"]:
        ts.add(
            ts_key.format(card_id=pricing["card_id"], currency="usd"),
            int(pricing["date"].timestamp() * 1000),
            pricing["prices"]["usd"],
            on_duplicate="LAST",
        )
    if pricing["prices"]["usd_foil"]:
        ts.add(
            ts_key.format(card_id=pricing["card_id"], currency="usd_foil"),
            int(pricing["date"].timestamp() * 1000),
            pricing["prices"]["usd_foil"],
            on_duplicate="last",
        )
    if pricing["prices"]["tix"]:
        ts.add(
            ts_key.format(card_id=pricing["card_id"], currency="tix"),
            int(pricing["date"].timestamp() * 1000),
            pricing["prices"]["tix"],
            on_duplicate="last",
        )
    if pricing["prices"]["eur"]:
        ts.add(
            ts_key.format(card_id=pricing["card_id"], currency="eur"),
            int(pricing["date"].timestamp() * 1000),
            pricing["prices"]["eur"],
            on_duplicate="last",
        )
    if pricing["prices"]["eur_foil"]:
        ts.add(
            ts_key.format(card_id=pricing["card_id"], currency="eur_foil"),
            int(pricing["date"].timestamp() * 1000),
            pricing["prices"]["eur_foil"],
            on_duplicate="last",
        )
    if pricing["prices"]["usd_etched"]:
        ts.add(
            ts_key.format(card_id=pricing["card_id"], currency="usd_etched"),
            int(pricing["date"].timestamp() * 1000),
            pricing["prices"]["usd_etched"],
            on_duplicate="last",
        )

449824it [00:44, 10047.98it/s]


KeyboardInterrupt: 